# Automatic Lab Evaluator

## Assessment based on student-provided results

Version History:

Version 0.1 - Jerónimo Arenas García, Jesús Cid Sueiro, Vanessa Gómez Verdejo, Dec. 2016

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
from os.path import isfile, join
import scipy.io as sio
import scipy
import zipfile as zp
import shutil
import difflib

## 1. Read datafiles for all students

Student datafiles can be in any of the following formats:

   * `'.zip'`: When uncompressed, the zip may contain one or several matlab/numpy files. All matlab/numpy files are read and incorporated to a pandas Dataframe where each student is a column, and each index is a variable available for the exam solution
   * `.mat'`: All data variables for the students are given in a single matlab file
   * `.npz'`: All data variables for the students are given in a single numpyº file

In [2]:
def readdatafiles(datafiles_path, splitsymbol):

    temporary_dir = './tmp'
    df = pd.DataFrame()
    
    datafiles = [f for f in os.listdir(datafiles_path) if isfile(join(datafiles_path, f))]
    for dtfile in datafiles:
        if dtfile.endswith('zip'):
            #All files will be extracted and the contents incorporated to the Dataframe
            NIA = dtfile.split(splitsymbol)[0]
            #print NIA
            idx = []
            val = []
            zpobj = zp.ZipFile(join(datafiles_path, dtfile))
            for fileinzip in zpobj.namelist():
                #Remove files beginning with '_' that may be hidden OS files 
                if (not fileinzip.startswith('_')) and (fileinzip.endswith('mat') or fileinzip.endswith('npz')):
                    #Matlab/NPZ files are extracted to a temporal subfolder
                    zpobj.extract(fileinzip, temporary_dir)
                    if fileinzip.endswith('mat'):
                        data = sio.loadmat(join(temporary_dir,fileinzip))
                    else:
                        data = np.load(join(temporary_dir,fileinzip))
                        if data.keys()==['arr_0']:
                            data = data['arr_0'].tolist()
                    #Read all variable names and the corresponding data values
                    for var in [el for el in data.keys() if not el.startswith('_')]:
                        idx.append(var)
                        val.append(data[var])
            #If
            if idx:
                #print idx
                #df = [df, pd.Series(val,index=idx, name=NIA)]
                df = pd.concat([df, pd.Series(val,index=idx, name=NIA)], axis=1)
                #df[NIA] = pd.Series(val,index=idx)
                #print pd.Series(val,index=idx)
                #print df[NIA]
                
            #Remove temporary directory, if it has been created
            if os.path.exists(temporary_dir):
                shutil.rmtree(temporary_dir)
                    
        elif dtfile.endswith('mat') or dtfile.endswith('npz'):
            NIA = dtfile.split(splitsymbol)[0]
            #print NIA
            idx = []
            val = []
            if dtfile.endswith('mat'):
                data = sio.loadmat(join(datafiles_path, dtfile))
            else:
                data = np.load(join(datafiles_path, dtfile))
            #Read all variable names and the corresponding data values
            for var in [el for el in data.keys() if not el.startswith('_')]:
                idx.append(var)
                val.append(data[var])
                
            if idx:
                df = pd.concat([df, pd.Series(val,index=idx, name=NIA)], axis=1)
                        
    return df
        

In [3]:
###########################################
# EXAM DEPENDENT VARIABLE
datafiles_path = '../GenerateData/'
##########################################

student_data = readdatafiles(datafiles_path, splitsymbol='_')

print 'Number of students in dataframe: ' + str(student_data.shape[1])
print 'Number of variables read: ' + str(student_data.shape[0])

print 'Displaying data for first three students ... '
student_data[student_data.columns[:3]]

Number of students in dataframe: 6
Number of variables read: 8
Displaying data for first three students ... 


,100275003,100291751,100291918
str_reg,"[[2.37675920113], [0.951469753966], [3.5027663...","[[-0.0288747123478], [7.92282144655], [1.05294...","[[-1.18191995385], [-4.36867911082], [-4.25361..."
sval_reg,"[[3.85445555957], [2.34222337967], [4.35809280...","[[4.81703835638], [0.387573160177], [14.001584...","[[-1.23648876408], [-3.84656562627], [-2.65649..."
xtr_reg,"[[0.839057585875, 0.589670389279], [0.55985250...","[[-1.21162330729, -0.33919274492], [1.57089660...","[[-0.435888878118, 1.26351544807], [-1.5059069..."
xtr_rl,"[[0.368589633828, 0.430068779351, 0.0664804478...","[[0.331881469126, 0.602033652044, 0.6459483987...","[[-0.0375187315887, 0.891688919951, 0.17435354..."
xval_reg,"[[-2.1100926606, -0.57344524891], [-0.16425731...","[[1.33811140075, 0.768747291224], [-0.39686418...","[[0.782647132783, -0.317493309749], [1.5443957..."
xval_rl,"[[-0.342953578835, 0.00696465776527, 0.2480226...","[[0.682291482678, -0.226123607538, -1.35449744...","[[-0.582304029364, 0.397088105521, 0.475633083..."
ytr_rl,"[[1], [1], [1], [0], [0], [1], [1], [1], [0], ...","[[1], [1], [1], [0], [0], [1], [1], [0], [0], ...","[[1], [0], [1], [0], [1], [1], [1], [0], [0], ..."
yval_rl,"[[0], [1], [1], [0], [0], [1], [1], [0], [0], ...","[[0], [1], [0], [1], [0], [1], [0], [0], [1], ...","[[1], [0], [1], [1], [1], [0], [1], [0], [0], ..."


## 2. Read answers provided by students


### 2.1. Read student results into panda dataframe

In [ ]:
###########################################
# EXAM DEPENDENT VARIABLE
results_path = '../Entregas/'
#Requested variable names in the wording of the exam
truenames = ['s0', 'E2val', 'E2val_knn', 'bestK', 'w_mean', 'w_cov', 'logM',
            'xtr_n','xval_n','w10', 'e10', 'en', 'eopt', 'mopt', 'wopt']
###########################################

student_results = readdatafiles(results_path, splitsymbol='_')
newindex = truenames+[el for el in student_results.index.tolist() if el not in truenames]
student_results = student_results.reindex(newindex)

print 'Number of students in dataframe: ' + str(student_results.shape[1])
print 'Number of variables read: ' + str(student_results.shape[0])

print 'Displaying data for first three students ... '
student_results[student_results.columns[:13]]

### 2.2. Common Mistakes on variable names

In view of all variable names provided by all students, we may decide to allow alternative names for variables without any penalty

In [5]:
print 'Number of students in dataframe: ' + str(student_results.shape[1])

print '\nDisplaying number of missing data per variable name. \nThose with a large number are a potential common mistake\nfor a variable name'

student_results.isnull().sum(axis=1)

Number of students in dataframe: 5

Displaying number of missing data per variable name. 
Those with a large number are a potential common mistake
for a variable name


s0            0
E2val         0
E2val_knn     1
bestK         1
w_mean        0
w_cov         0
logM          3
xtr_n         0
xval_n        0
w10           3
e10           3
en            4
eopt          3
mopt          3
wopt          3
E2val_knn:    4
bestk         4
str_reg       4
sval_reg      4
xtr_reg       4
xtr_rl        4
xval_reg      4
xval_rl       4
ytr_rl        4
yval_rl       4
dtype: int64

In [ ]:
###########################################
# EXAM DEPENDENT VARIABLE

#Dictionary with accepted mistakes in the following format
#  Expected variable name : Accepted mistake
Mistakes = {'E2val_knn': 'E2val_knn:',
            'bestK': 'bestk'};
##########################################

for el in Mistakes:
    student_results.loc[el] = student_results.loc[el].fillna(student_results.loc[Mistakes[el]])
    
for el in student_results.index.tolist():
    if el not in truenames:
        student_results.drop(el, inplace=True)
        
student_results[student_results.columns[:5]]

### 2.3. Name to NIA dictionary

Finally, since datafiles are created by NIA and results are available per student name, we need to create a dictionary connecting them

In [7]:
###########################################
# EXAM DEPENDENT VARIABLE
excel_file = 'ListaClase.xlsx'
language = 'english'
###########################################

student_NIA_names = pd.read_excel(excel_file)

#UTF-8 encoding of everything
for fld in student_NIA_names.keys():
    if fld != 'NIU':
        student_NIA_names[fld] = student_NIA_names[fld].str.encode('utf8')

NIA_name = {}

for el in student_results.columns.tolist():

    sim_list = []

    for idx,NIA in enumerate(student_NIA_names['NIU'].values):
    
        if language=='english':
            std_name = student_NIA_names['First name'].values.tolist()[idx] + ' ' + \
                            student_NIA_names['Surname'].values.tolist()[idx]
            sim_list.append(difflib.SequenceMatcher(a=el.lower(), b=std_name.lower()).ratio())
        else:
            std_name = student_NIA_names['Nombre'].values.tolist()[idx] + ' ' + \
                            student_NIA_names['Apellido(s)'].values.tolist()[idx]
            sim_list.append(difflib.SequenceMatcher(a=el.lower(), b=std_name.lower()).ratio())
    
    max_sim = max(sim_list)
    max_idx = sim_list.index(max_sim)
    
    NIA_name[student_NIA_names['NIU'].values.tolist()[max_idx]] = el

#Create dictionary for students that did not hand in anything
NIA_name_nodata = {}
if language=='english':
    lista1 = [el for el in student_NIA_names['NIU'].values if el not in NIA_name.keys()]
    lista2 = [student_NIA_names[student_NIA_names['NIU']==NIA]['First name'].values.tolist()[0] + ' ' + \
          student_NIA_names[student_NIA_names['NIU']==NIA]['Surname'].values.tolist()[0] for NIA in lista1]   
else:
    lista1 = [el for el in student_NIA_names['NIU'].values if el not in NIA_name.keys()]
    lista2 = [student_NIA_names[student_NIA_names['NIU']==NIA]['Nombre'].values.tolist()[0] + ' ' + \
          student_NIA_names[student_NIA_names['NIU']==NIA]['Apellido(s)'].values.tolist()[0] for NIA in lista1]   

for el in zip(lista1,lista2):
    NIA_name_nodata[el[0]] = el[1]
    
#Create name to NIA dictionary
name_NIA = {NIA_name[el]: el for el in NIA_name}
name_NIA_nodata = {NIA_name_nodata[el]: el for el in NIA_name_nodata}


In [ ]:
#print name_NIA
#print NIA_name
for el in NIA_name.keys():
    print str(el) + ' : ' + NIA_name[el]


At this point we have:

   * student_data: dataframe with data given to the students. Each index is a variable, and each column a NIA
   * student_results: dataframe with student results. Each index is a variable, and each column a name
   * NIA_name: NIA to name dictionary
   * name_NIA: name to NIA dictionary

## 3. Exam solution

In this section we implement the solution to the exam. This is a function that takes the variables generated for a given student and the answers provided by the student, and generates a structure with all posible answers, possibly with a penalty term associated to each answer.

In [ ]:
#Busqueda por nombre
student = 
print student_data[str(name_NIA[student])]
print " "
print student_results[student.decode('utf8')]
##Busqueda por NIA
#NIA = '100339092'
#print student_data[NIA]
#print " "
#print student_results[NIA_name[int(NIA)]]

In [10]:
### Some methods required for the Classification solution
def normalize(X, mx=None, sx=None):

    # Compute means and standard deviations
    if mx is None:
        mx = np.mean(X, axis=0)
    if sx is None:
        sx = np.std(X, axis=0)

    # Normalize
    X0 = (X-mx)/sx

    return X0, mx, sx

# Define the logistic function
def logistic(x):
    p = 1.0 / (1 + np.exp(-x))
    return p

# MAP trainer.
def logregFitR(Z_tr, Y_tr, rho, C, n_it):

    # Initialize variables
    n_dim = Z_tr.shape[1]
    nll_tr = np.zeros(n_it)
    w = 0*np.random.randn(n_dim, 1)

    # Running the gradient descent algorithm
    for n in range(n_it):

        # Compute posterior probabilities for weight w
        p1_tr = logistic(np.dot(Z_tr, w))
        p0_tr = logistic(-np.dot(Z_tr, w))
        # Compute negative log-likelihood
        nll_tr[n] = (- np.dot(Y_tr.T, np.log(p1_tr)) - np.dot((1-Y_tr).T, np.log(p0_tr)))
        # Update weights
        w = (1-2*rho/C)*w + rho*np.dot(Z_tr.T, Y_tr - p1_tr)

    return w, nll_tr

# MAP trainer.
def computeNLL(Z, Y, w):

    # Compute posterior probabilities for weight w
    p1 = logistic(np.dot(Z, w))
    p0 = logistic(-np.dot(Z, w))
    # Compute negative log-likelihood
    nll = (- np.dot(Y.T, np.log(p1)) - np.dot((1-Y).T, np.log(p0)))

    return nll

# Compute predictions for a given model
def logregPredict(Z, w):

    # Compute posterior probability of class 1 for weights w.
    p = logistic(np.dot(Z, w))
    # Classify
    D = [int(round(pn)) for pn in p]

    return p, D

In [31]:
from sklearn import neighbors
from math import pi

def isVarFail(x):
    
    # Check if the variable contains all None.
    try:
        return np.all(np.isnan(x))
    except:
        return True

def SolveClassif(Xtrain, Xval, ytr_rl, yval_rl, wst=None):

    # # Sec. B
    # Set parameters
    C = 100
    n_it = 500
    rho = 0.001

    # Compute extended vector
    n_tr = Xtrain.shape[0]
    Z_tr = np.concatenate((np.ones((n_tr, 1)), Xtrain), axis=1)
    if wst is None:
        w10_, nll = logregFitR(Z_tr, ytr_rl, rho, C, n_it)
    else:
        w10_ = wst

    n_val = Xval.shape[0]
    Z_val = np.concatenate((np.ones((n_val, 1)), Xval), axis=1)
    p1, D1 = logregPredict(Z_val, w10_)
    e10_ = np.mean(np.array(D1)[:, np.newaxis] != yval_rl)

    # # Sec. C
    # Removing single variables
    wsum = np.cumsum(w10_[::-1])[::-1]
    wm = wsum * (wsum >= 0.1 * np.sum(w10_))
    n_val = Xval.shape[0]
    Z_val = np.concatenate((np.ones((n_val, 1)), Xval), axis=1)
    p1, D1 = logregPredict(Z_val, wm)
    en_ = np.mean(np.array(D1)[:, np.newaxis] != yval_rl)

    absw10_ = np.abs(w10_)
    wsum = np.cumsum(absw10_[::-1])[::-1]
    wm = wsum * (wsum >= 0.1 * np.sum(absw10_))
    n_val = Xval.shape[0]
    Z_val = np.concatenate((np.ones((n_val, 1)), Xval), axis=1)
    p1, D1 = logregPredict(Z_val, wm)
    en_b = np.mean(np.array(D1)[:, np.newaxis] != yval_rl)

    
    eopt_ = 10000*n_val
    for i in range(10):

        cols = range(i + 1)
        Ztr_i = Z_tr[:, cols]
        Zval_i = Z_val[:, cols]

        wi, nll = logregFitR(Ztr_i, ytr_rl, rho, C, n_it)
        p1, D1 = logregPredict(Zval_i, wi)

        ei = np.mean(np.array(D1)[:, np.newaxis] != yval_rl) + 0.005 * i**2

        if ei <= eopt_:
            eopt_ = ei
            mopt_ = i
            wopt_ = wi

    return w10_, e10_, en_, en_b, eopt_, mopt_, wopt_
    
def SolveLabXX(data, st_solution):
    """Solver for the practical
    Input parameters:
    data: A series with the data given to the student
    st_solution: The solution provided by the student
    
    Output: A dataseries where each element is a list of tuples
    with the format [(solution1, factor1), (solution2, factor2)]
    
    Factors are multiplicative factors to account for possible
    penalties. A factor 1 should be given to a solution that should
    not be penalized.
    """
    
    ds_values = []
    ds_indexes = []
    
    # #############
    # ## REGRESSION

    ## Sec. 2.1
    s0 = []
    
    s0.append((np.mean(data['str_reg']),1))
    
    ds_values.append(s0)
    ds_indexes.append('s0')
    
    E2val = []
    
    
    E2val.append((np.mean((data['sval_reg'] - s0[0][0])**2),1))
    #Penalizamos al 50% si hicieron el total, en lugar del promedio
    E2val.append((np.sum((data['sval_reg'] - s0[0][0])**2),.5))
    
    #Usando el valor de s0 proporcionado por el alumno
    st_s0 = np.array(st_solution['s0']).flatten()[0]
    if not np.all(np.isnan(st_s0)):
        E2val.append((np.mean((data['sval_reg'] - st_s0)**2),.7))
        E2val.append((np.sum((data['sval_reg'] - st_s0)**2),.35))
    
    ds_values.append(E2val)
    ds_indexes.append('E2val')

    ## Sec. 2.2
    E2val_knn = []
    bestK = []
    
    values = []
    for k in range(10):
        knn = neighbors.KNeighborsRegressor(k+1, weights='uniform')
        sval = knn.fit(data['xtr_reg'], data['str_reg']).predict(data['xval_reg'])
        values.append(np.mean((data['sval_reg'] - sval)**2))
    values = np.array(values)
    E2val_knn.append((values, 1))
    bestK.append((np.argmin(values)+1, 1))
    #x0.5 if the index is given instead of the value of K
    bestK.append((np.argmin(values), 0.5))

    #We repeat the exercise averaging the predictions using distance in knn
    values = []
    for k in range(10):
        knn = neighbors.KNeighborsRegressor(k+1, weights='distance')
        sval = knn.fit(data['xtr_reg'], data['str_reg']).predict(data['xval_reg'])
        values.append(np.mean((data['sval_reg'] - sval)**2))
    values = np.array(values)
    E2val_knn.append((values, 1))
    bestK.append((np.argmin(values)+1, 1))
    #x0.5 if the index is given instead of the value of K
    bestK.append((np.argmin(values), 0.5))

    ds_values.append(E2val_knn)
    ds_indexes.append('E2val_knn')
    ds_values.append(bestK)
    ds_indexes.append('bestK')


    ## Sec. 2.3
    Z = np.hstack((np.ones((500,1)),data['xtr_reg'],np.exp(data['xtr_reg'])))

    Sigma_p = 2*np.eye(5)
    var_n = .5

    w_cov = []
    w_cov.append((np.linalg.inv(Z.T.dot(Z)/var_n + np.linalg.inv(Sigma_p)),1))

    ds_values.append(w_cov)
    ds_indexes.append('w_cov')

    w_mean = []
    w_mean.append((w_cov[0][0].dot(Z.T).dot(data['str_reg'])/var_n,1))

    ds_values.append(w_mean)
    ds_indexes.append('w_mean')
    
    ## Sec. 2.4
    logM = [] 
    logM_a = -(Z.shape[1]*np.log(2*pi)/2)-np.log(np.linalg.det((2)*Z.dot(Z.T) + (0.5)*np.eye(500)))/2 -data['str_reg'].T.dot(np.linalg.inv((2)*Z.dot(Z.T) + (0.5)*np.eye(500))).dot(data['str_reg'])/2

    logM.append((logM_a, 1))
    
    ds_values.append(logM)
    ds_indexes.append('logM')
 
    # #################
    # ## CLASSIFICATION

    ## Standard correct response:

    ## Sec. 3a
    # Get data
    xtr_rl = data['xtr_rl']
    xval_rl = data['xval_rl']
    ytr_rl = data['ytr_rl']
    yval_rl = data['yval_rl']

    # Compute mean and std and normalize
    Xtrain, mx0, sx0 = normalize(xtr_rl)
    Xval, mx0, sx0 = normalize(xval_rl, mx0, sx0)

    ## Sec. 3a
    # Normalized variables have been computed in the previous section
    xtr_n = [(Xtrain, 1)]
    xval_n = [(Xval, 1)]

    # Solve the rest of the questions
    w10_, e10_, en_, en_b, eopt_, mopt_, wopt_ = SolveClassif(Xtrain, Xval, ytr_rl, yval_rl, wst=None)
    w10 = [(w10_, 1)]
    e10 = [(e10_, 1)]
    en = [(en_, 1), (en_b, 1)]
    eopt = [(eopt_, 1)]
    mopt = [(mopt_, 1)]
    wopt = [(wopt_, 1)]

    ## ALTERNATIVE 2: USING STUDENT'S TRAINING DATA MATRIX
    # From now on, I use the declared rho
    if not isVarFail(st_solution['xtr_n']):
        Xtrain2 = st_solution['xtr_n']
        if np.array_equal(Xtrain.shape, Xtrain2.shape):
            w10_2, e10_2, en_2, en_b2, eopt_2, mopt_2, wopt_2 = SolveClassif(Xtrain2, Xval, ytr_rl, yval_rl, wst=None)
            w10 += [(w10_2, 1)]
            e10 += [(e10_2, 1)]
            en += [(en_2, 1), (en_b2, 1)]
            eopt += [(eopt_2, 1)]
            mopt += [(mopt_2, 1)]
            wopt += [(wopt_2, 1)]
            
    ## ALTERNATIVE 3: USING STUDENT'S WEIGHTS
    w10st = np.array(st_solution['w10'])
    if not isVarFail(w10st):
        if np.array_equal(w10_.shape, w10st.shape):
            w10_3, e10_3, en_3, en_b3, eopt_3, mopt_3, wopt_3 = SolveClassif(Xtrain2, Xval, ytr_rl, yval_rl, w10st)
            e10 += [(e10_3, 1)]
            en += [(en_3, 1), (en_b3, 1)]

    ## ALTERNATIVE 4: USING STUDENT'S WEIGHTS AND DATA VALIDATION MATRIX
    # Now the same, but using the student weights and data matrix
    if not isVarFail(w10st) and not isVarFail(st_solution['xval_n']):
        Xval2 = st_solution['xval_n']
        if np.array_equal(Xval.shape, Xval2.shape) and np.array_equal(w10_.shape, w10st.shape):
            w10_4, e10_4, en_4, en_b4, eopt_4, mopt_4, wopt_4 = SolveClassif(Xtrain, Xval2, ytr_rl, yval_rl, w10st)
            e10 += [(e10_4, 1)]
            en += [(en_4, 1), (en_b4, 1)]

    ds_indexes.append('xtr_n')
    ds_values.append(xtr_n)
    ds_indexes.append('xval_n')
    ds_values.append(xval_n)
    ds_indexes.append('w10')
    ds_values.append(w10)
    ds_indexes.append('e10')
    ds_values.append(e10)    
    ds_indexes.append('en')
    ds_values.append(en)
    ds_indexes.append('eopt')
    ds_values.append(eopt)
    ds_indexes.append('mopt')
    ds_values.append(mopt)
    ds_indexes.append('wopt')
    ds_values.append(wopt)

    return pd.Series(ds_values, ds_indexes)
    

    
    return pd.Series(ds_values, ds_indexes)
    

## 4. Evaluation of all students

In [32]:
def automatic_evaluator(NIA, student_results, solution, weights, tolerance):
    
    val=[]
    idx=[]
    
    if len(solution.keys())==len(weights) and len(solution.keys())==len(tolerance):

        for wgh, el, tol in zip(weights,solution.keys(),tolerance):

            var_summary = []
            #If the student has delivered the variable, append 1; otherwise 0
            try:
                isAllNaN = np.all(np.isnan(student_results[el]))
            except:
                isAllNaN = True
                
            if not isAllNaN:    # np.all(np.isnan(student_results[el])):
                var_summary.append(1)
                #Check all possible solutions against the one provided by the student
                factors = [entry[1] for entry in solution[el]
                           if np.array_equal(np.array(student_results[el]).flatten().shape,
                                             np.array(entry[0]).flatten().shape) and
                              np.mean(np.abs(np.array(entry[0]).flatten() - 
                                      np.array(student_results[el]).flatten())) < tol]
                print el
                #print [np.mean(np.abs(np.array(entry[0]).flatten() - 
                #                      np.array(student_results[el]).flatten())) for entry in solution[el]]
                print factors
                
                if len(factors):
                    max_factor = max(factors)
                    var_summary.extend([max_factor, wgh, max_factor*wgh])
                else:
                    var_summary.extend([0, wgh, 0])
            else:
                var_summary.extend([0, 0, wgh, 0])

            # Keep values corresponding to current variable
            val.append(var_summary)
            idx.append(el)
            
        final_score = sum([item[-1] for item in val])
        val.append(final_score)
        idx.append('FinalScore')
        
    else:
        print 'The number of weights and variables to evaluate differ. Please, check'
    
    val.append(NIA)
    idx.append('NIA')
    
    return pd.Series(val,index=idx)

In [ ]:
###########################################
# EXAM DEPENDENT VARIABLE
excel_output = 'Notas_MMC.xlsx'
all_students = 1 #Include in the list students that did not carry out the exam
weightsR = [.5, .5, 1, 0.5, 0.75, 0.75, 1]
weightsC = [.5, .5, .5, .5, 1, 1.0/3, 1.0/3, 1.0/3]
sR = sum(weightsR)
sC = sum(weightsC)
weights = [w*5.0/sR for w in weightsR] + [w*5.0/sC for w in weightsC]
print weights

tolerance = [1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2]
#tolerance = [1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 5e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2]
###########################################

df = pd.DataFrame()

print len(NIA_name.keys())

for NIA in NIA_name.keys():
    print NIA_name[NIA]
    solution = SolveLabXX(student_data[str(NIA)], student_results[NIA_name[NIA]])
    df[NIA_name[NIA].decode('utf8')] = automatic_evaluator(NIA, student_results[NIA_name[NIA]], solution, weights, 
                                                           tolerance)
print df

if all_students:
    for NIA in NIA_name_nodata.keys():
        df[NIA_name_nodata[NIA].decode('utf8')] = pd.Series([NIA],index=['NIA'])
        
df.T.to_excel(excel_output,columns=df.T.columns)